## 講習2 --- IRAFで1次処理

IRAFのタスクを使って、**１次処理**(生データからバイアスを引き、それをフラットで割る) を行ってみましょう。  
ここでは、IRAFの基本タスク、**imarith**, **imcombine**を使います。  

### ドームフラットでターゲットのフレームをフラット処理する (簡易版)

講習1で扱った'./data1/SUPA00317705.fits' はtarget1を観測した生データです。
これを、バイアス値を引いたあとに、フラットで割ることで、CCDの感度ムラと光学系の透過率のムラを補正します。
視野全体で一様な強度で光っている(と考えている)ものを観測してフラットを作成します。
(最近の)CCDではダークを生データから引くことはしません。(地上観測の近赤外アレイでは、ダークを引いてフラットで割り、スカイバイアスを引くという処理が必要になります。)

まず、このフラットを作成する必要があります。
ドームフラットの生データからバイアス値を引いたのちに**規格化**(メジアン値で割る)してフラットを作成します。
もし、CCDの感度ムラがなく、光学系の透過率も完全に一様であれば、フラットは全てのピクセルで1.0の値をもちます。
通常は複数のフラットの平均から、より尤もらしいフラットを作成します。  

下ではまず、ドームフラット1枚だけからフラット作成します。**imstat**と**imarith**を使用します。  
次に、ドームフラット7枚からフラット作成します。**imcombine**を使用します。  

** ドームフラット1枚だけを使う **   

'./data1/SUPA00317505.fits' はドームフラットのBバンドの生データです。
CCDのフレームには、観測した光に加えて、X方向に一様なバイアス値が加算されています。
この**簡易版**の処理では、そのバイアス値がY方向にも一様として処理をします。(のちにY方向の依存も考慮に入れた手法を紹介します。)
そのバイアス値をオーバースキャン領域から推定します。Suprime Camの5番フレームでは2049列目あたりから右側がオーバースキャン領域です。

In [2]:
from pyraf import iraf

In [3]:
iraf.unlearn('imstat')
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.nclip = 3

In [4]:
iraf.imstat('./data1/SUPA00317505.fits[2049:2080, *]')

#     MIDPT      MEAN    STDDEV
      9984.     9984.     5.046


ドームフラットの光があたっている部分のメジアンを求めておきます。

In [5]:
iraf.imstat('./data1/SUPA00317505.fits[1:2048, *]')

#     MIDPT      MEAN    STDDEV
     18933.    18928.     210.1


**imarith**の出番です。バイアス値を引いてから、メジアン値で規格化してフラットを作成しましょう。

In [6]:
iraf.imarith('./data1/SUPA00317505.fits', '-' , 9984, 'bflatn5a.fits')  
iraf.imarith('bflatn5a.fits', '/' , 8949, 'bflatn5a.fits')  #  18933 - 9984 = 8949

このフラットでターゲットの生データを割ります。このときも、まず、生データからバイアス値を引きます。

In [7]:
iraf.imstat('./data1/SUPA00317705.fits[2049:2080, *]')

#     MIDPT      MEAN    STDDEV
      9989.     9989.     4.841


In [8]:
iraf.imarith('./data1/SUPA00317705.fits', '-', 9989, 'btarget1n5a.fits')
iraf.imarith('btarget1n5a.fits', '/', 'bflatn5a.fits', 'btarget1n5a.fits')

これでできました。 btarget1n5a.fitsをds9で表示して確かめてみましょう。  


**ドームフラット7枚を使う**   
**iraf.imcombine**の出番です。  
'./data1/SUPA003175[0-6]5.fits' はBバンドのドームフラットです。


In [9]:
import glob # pythonの組み込みモジュール。ワイルドカードを使ったファイル処理など。

In [10]:
flist = glob.glob('./data1/SUPA003175[0-6]5.fits')

In [11]:
print(flist)

['./data1/SUPA00317505.fits', './data1/SUPA00317515.fits', './data1/SUPA00317525.fits', './data1/SUPA00317535.fits', './data1/SUPA00317545.fits', './data1/SUPA00317555.fits', './data1/SUPA00317565.fits']


forループを使って、このリストからファイルを一つづつimstatに入力します。

In [12]:
for img in flist:
    iraf.imstat(img + '[2049:2080, *]')

#     MIDPT      MEAN    STDDEV
      9984.     9984.     5.046
#     MIDPT      MEAN    STDDEV
      9983.     9983.     5.039
#     MIDPT      MEAN    STDDEV
      9983.     9983.     5.036
#     MIDPT      MEAN    STDDEV
      9983.     9983.     5.032
#     MIDPT      MEAN    STDDEV
      9983.     9983.     5.038
#     MIDPT      MEAN    STDDEV
      9983.     9983.     5.056
#     MIDPT      MEAN    STDDEV
      9983.     9983.     5.071


上では、glob.globで抽出したリストをいったん変数に代入しましたが、以下のように直接forループに入れても大丈夫です。

In [13]:
iraf.imstat.fields = 'midpt'  #  どうせメジアンしか使わない

for img in glob.glob('./data1/SUPA003175[0-6]5.fits'):
    out1 = iraf.imstat(img + '[2049:2080, *]', format='no', Stdout=1)
    out2 = iraf.imstat(img + '[1:2048, *]', format='no', Stdout=1)
    print (out1, out2)

['9983.81'] ['18932.82']
['9983.021'] ['18950.5']
['9982.991'] ['18963.1']
['9983.005'] ['19029.06']
['9982.896'] ['18995.43']
['9982.938'] ['18935.77']
['9982.917'] ['18969.67']


out1, out2は、それぞれ、1つしか要素を持たないリストとして得られました。  
下のように、リストの最初の要素を抽出することで値を得ることができます。ただし、文字列です。

In [14]:
out1[0]

'9982.917'

**float()**関数で数値(浮動小数点数)に変換してやります。

In [15]:
float(out1[0])

9982.917

それでは、各ドームフラットからフラットを作成し、それらをメジアンでコンバインします。

In [16]:
iraf.imstat.fields = 'midpt'  

num = 0
comstr = ''
for img in glob.glob('./data1/SUPA003175[0-6]5.fits'):
    
    out1 = iraf.imstat(img + '[2049:2080, *]', format='no', Stdout=1)  #  オーバースキャン領域
    out2 = iraf.imstat(img + '[1:2048, *]', format='no', Stdout=1)  #  光のあたってる領域
    med1 = float(out1[0])  #  文字列を数値に変換
    med2 = float(out2[0])
    
    nflat =  'tmp' + str(num) + '.fits'   #  それぞれのフラットを作成
    iraf.imarith(img, '-', med1, nflat)  #  バイアス値をひく
    iraf.imarith(nflat, '/', med2 - med1, nflat) #  バイアスを考慮して規格化
    
    num += 1
    comstr += nflat + ','  #  imcombineの引数として与えるための文字列

print (comstr)  #  なぜ下でcomstr[:-1]と、末尾の一文字を削除するか理解するためにあえて表示 
    
iraf.imcombine(comstr[:-1], 'bflatn5.fits', combine='median')
iraf.imdelete(comstr[:-1])  #  中間ファイルを削除。お掃除お掃除。

tmp0.fits,tmp1.fits,tmp2.fits,tmp3.fits,tmp4.fits,tmp5.fits,tmp6.fits,

Aug 21 13:22: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
              tmp0.fits
              tmp1.fits
              tmp2.fits
              tmp3.fits
              tmp4.fits
              tmp5.fits
              tmp6.fits

  Output image = bflatn5.fits, ncombine = 7


これでドームフラットを7枚使ったフラットができました。  
生データをこれで処理してやります。

In [17]:
iraf.imarith('./data1/SUPA00317705.fits', '-', 9989, 'btarget1n5.fits')
iraf.imarith('btarget1n5.fits', '/', 'bflatn5.fits', 'btarget1n5.fits')

### 演習2 

**2-1.**  
別ターゲットtarget2を観測した、'./data1/SUPA00317885.fits' について、バイアス引き+フラット割りの処理をしましょう。
これは5番フレームです。フィルターも同じBバンドなので、フラット割りには、'bflatn5.fits'が使えます。  
この結果のフレームを'btarget2n5.fits'と呼ぶことにします。(後の演習で利用します)  

**2-2.**  
'./data1/SUPA00317705.fits'と同じ観測の2番フレームの生データ'./data2/SUPA00317702.fits' について、
バイアス引き+フラット割りの処理をしましょう。
先ほどの5番フレームとは違い、これは2番フレームなので、2番フレームのためのフラットを作成する必要があります。  
(1) './data2/SUPA00317502.fits' を規格化したものをフラットとして作成する。(1枚フラット)  
(2) './data2/SUPA003175[0-6]2.fits' から平均のフラットを作成する。  
(3) 上のどちらか(あるいは両方)のフラットを使って、バイアス引き後のフラット割りを行う。

注意 : 2番フレームはオーバースキャン領域が5番とは異なる。

### 補足 バイアス値のY方向依存も考慮に入れる

今回使用するフラットでは0.1 x 数パーセントの違いしかありませんが、Y方向の依存も考慮に入れた方法を紹介しておきます。
ドームフラットのデータを1枚だけ使うケースを例にします。
irafのblkavgを使って、オーバースキャン領域の各lineの算術平均を求めます。(本当はメジアンがよいが。)　　

In [18]:
iraf.blkavg('./data1/SUPA00317505.fits[2049:2080, *]', 'bias1.fits', 32, 1)

bias1.fitsはサイズが(1,4100)の1次元データです。これをX方向に2080倍のばします。

In [19]:
iraf.blkrep('bias1.fits', 'bias2.fits', 2080)

このバイアスを生データから引いてやります。

In [20]:
iraf.imarith('./data1/SUPA00317505.fits', '-' , 'bias2.fits', 'bflatn5by.fits')

In [21]:
iraf.imstat('bflatn5by.fits[1:2048, *]', fields='midpt')

#     MIDPT
      8957.


In [22]:
iraf.imarith('bflatn5by.fits', '/', 8957, 'bflatn5by.fits')

バイアスのY方向依存も考慮に入れたフラットができました。  
次にターゲットのフレームをフラットで割ります。  
ここでもバイアス値のY方向依存を考慮にいれます。  

In [23]:
iraf.blkavg('./data1/SUPA00317705.fits[2049:2080, *]', 'bias1t.fits', 32, 1)
iraf.blkrep('bias1t.fits', 'bias2t.fits', 2080)
iraf.imarith('./data1/SUPA00317705.fits', '-' , 'bias2t.fits', 'btarget1n5by.fits')
iraf.imarith('btarget1n5by.fits', '/', 'bflatn5by.fits', 'btarget1n5by.fits')